In this example we will create a custom callback that will log the weights and the gradients of the parameters using Tensorboard.

#### Import Statements

#### Additional libraries

First of all we need to install the `tensorflow` and `tensorboard` libraries in order to use Tensorboard.

In [ ]:
! pip install tensorflow
! pip install tensorboard


# Custom Callback (Tensorboard)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from pytorch_wrapper.training_callbacks import AbstractCallback


#### CallBack definition

PyTorchWrapper enables us to inject custom buisness logic in several places of the
training process. In order to do that we need to create a Class that derives
from `pytorch_wrapper.training_callbacks.AbstractCallback` and implement the 
appropriate method(s).

The possible methods are the following:
- on_training_start: Called at the beginning of the training process.
- on_training_end: Called at the end of the training process.
- on_evaluation_start: Called at the beginning of the evaluation step.
- on_evaluation_end: Called at the end of the evaluation step.
- on_epoch_start: Called at the beginning of a new epoch.
- on_epoch_end: Called at the end of an epoch.
- on_batch_start: Called just before processing a new batch.
- on_batch_end: Called after a batch has been processed.
- post_predict: Called just after prediction during training time.
- post_loss_calculation: Called just after loss calculation.
- post_backward_calculation: Called just after backward is called.
- pre_optimization_step: Called just before the optimization step.


In [ ]:
class TensorBoardCallBack(AbstractCallback):
    def __init__(self, log_dir):
        super(TensorBoardCallBack, self).__init__()
        self.writer = SummaryWriter(log_dir=log_dir)
        self.n_iter = 0
        self.n_iter_real = 0

    def pre_optimization_step(self, training_context):
        """
        Called just before the optimization step.
        :param training_context: Dict containing information regarding the training process.
        """

        model = training_context['system'].model
        self.n_iter_real += 1

        if self.n_iter_real % 100 == 0:

            self.n_iter += 1

            for name, param in model.named_parameters():
                if param.requires_grad:
                    self.writer.add_histogram(name, param.detach().cpu().numpy(), self.n_iter)

            for name, param in model.named_parameters():
                if param.requires_grad:
                    self.writer.add_histogram(name + '-grad', param.grad.detach().cpu().numpy(), self.n_iter)


Now we can create this callback and pass it to the `train` method of a `System` object. After training you will be able 
navigate to the log_dir path and excecute `tensorboard` in order to see the histograms of the weights and gradients that where computed during training.